In [1]:
import pandas as pd 
import numpy as np 
import h5py, sys, traceback, os, tqdm, time
from script.utilize import pdgid, purity_classifier
import multiprocessing as mp

In [2]:
NN_RESULT = "./data/results_2020_10_17_nofilter.npz"

results = np.load(NN_RESULT)
full_dataset = results["dataset"]
full_predictions = results["predictions"]
full_targets = results["targets"]
n_match = results["n_match"]
num_samples = full_predictions.shape[0]

num_jets = (full_dataset.sum(-1) > 0).sum(-1)

In [3]:
chi2_storage = []
delta_R_storage = []
NUM_CHI2 = 59

for i in tqdm.trange(NUM_CHI2):
    with h5py.File(f"./backup/evaluate_new_with_JES/output_vector_{i + 1}.h5", 'r') as file:
        chi2_result = np.stack(file["Chi2_result"][:]).astype(np.int)
        delta_R_result = np.stack(file["Delta_R_result"][:]).astype(np.int)
    if i == 0:
        with h5py.File(f"./backup/evaluate_new_with_JES/event_recod_testing_with_chi2_new_{i + 1}.h5", 'r') as file:
            tmp_jet_pt = file["jet_pt"][:]
            N_match_top_in_event = file['N_match_top_in_event'][:]
    else: 
        with h5py.File(f"./backup/evaluate_new_with_JES/event_recod_testing_with_chi2_new_{i + 1}.h5", 'r') as file:
            tmp_jet_pt = np.concatenate((tmp_jet_pt, file["jet_pt"][:]))
            N_match_top_in_event = np.concatenate((N_match_top_in_event, file["N_match_top_in_event"][:]))
    chi2_storage.append(chi2_result)
    delta_R_storage.append(delta_R_result)

chi2_result = np.concatenate(chi2_storage)
delta_R_result = np.concatenate(delta_R_storage)
jet_pt = tmp_jet_pt
del chi2_storage, delta_R_storage

100%|██████████| 59/59 [00:08<00:00,  6.96it/s]


In [4]:
def compute(source, target):
    MODEL = 'ttbar'
    correct_left = np.zeros(len(target))
    correct_right = np.zeros(len(target))
    case = np.zeros(len(target))

    for i in tqdm.trange(len(target)):

        left_target = source[i][:3]
        right_target = source[i][3:]

        left_src = target[i][:3]    
        right_src = target[i][3:]  

        if np.sum(left_src < 0 ) >0:
            correct_left[i] = 4       
        if np.sum(right_src < 0) >0:
            correct_right[i] = 4

        if correct_left[i] != 4 and correct_right[i] != 4:
            correct_left[i], correct_right[i] = purity_classifier(target[i], source[i], "pair", MODEL)
        elif correct_left[i] != 4 and correct_right[i] == 4:
            correct_left[i] = purity_classifier(target[i], source[i], "left", MODEL)
        elif correct_left[i] == 4 and correct_right[i] != 4:
            correct_right[i] = purity_classifier(target[i], source[i], "right", MODEL)
        else:
            pass

        if correct_left[i] == 1 and correct_right[i] == 1:
            case[i] = 1
        elif correct_left[i] != 1 and correct_right[i] == 1 and correct_left[i] != 4 and correct_right[i] != 4:
            case[i] = 2
        elif correct_left[i] == 1 and correct_right[i] != 1 and correct_left[i] != 4 and correct_right[i] != 4:
            case[i] = 2
        elif  correct_left[i] != 1 and  correct_right[i] != 1 and correct_left[i] != 4 and correct_right[i] != 4:
            case[i] = 3
        elif correct_left[i] == 4 or  correct_right[i] == 4:
            case[i] = 4
    return case, correct_right, correct_left

In [5]:
NN_predict_2, NN_truth_2 = [], []
NN_predict_1, NN_truth_1 = [], []

for i in range(len(full_predictions)):
    if n_match[i] == 2:
        NN_predict_2.append(full_predictions[i])
        NN_truth_2.append(full_targets[i])   
    elif n_match[i] == 1:
        NN_predict_1.append(full_predictions[i])
        NN_truth_1.append(full_targets[i])   
        
chi2_predict_2, chi2_truth_2 = [], []
chi2_predict_1, chi2_truth_1 = [], []

for i in range(len(chi2_result)):
    if np.sum(delta_R_result[i]< 0) == 0:
        chi2_predict_2.append(chi2_result[i])
        chi2_truth_2.append(delta_R_result[i])   
    elif (np.sum(delta_R_result[i][:3]< 0) == 1 and np.sum(delta_R_result[i][3:]< 0) == 0) or (np.sum(delta_R_result[i][:3]< 0) == 0 and np.sum(delta_R_result[i][3:]< 0) == 1):
        chi2_predict_1.append(chi2_result[i])
        chi2_truth_1.append(delta_R_result[i]) 

In [6]:
NN_case, NN_correct_right, NN_correct_left = compute(full_predictions, full_targets)
NN_case_1, NN_correct_right_1, NN_correct_left_1 = compute(NN_predict_1, NN_truth_1)
NN_cas_2, NN_correct_right_2, NN_correct_left_2 = compute(NN_predict_2, NN_truth_2)

100%|██████████| 386971/386971 [00:08<00:00, 43703.90it/s]


In [7]:
chi2_case, chi2_correct_right, chi2_correct_left = compute(chi2_result, delta_R_result)
chi2_case_1, chi2_correct_right_1, chi2_correct_left_1 = compute(chi2_predict_1, chi2_truth_1)
chi2_cas_2, chi2_correct_right_2, chi2_correct_left_2 = compute(chi2_predict_2, chi2_truth_2)

100%|██████████| 387031/387031 [00:09<00:00, 40882.44it/s]


In [34]:
NN_event_purity = np.sum(NN_case==1)/( np.sum(NN_case==1) + np.sum(NN_case==2) + np.sum(NN_case==3))
NN_two_top_purity = ( np.sum(NN_correct_right_2 ==1) + np.sum(NN_correct_left_2==1)  )/( np.sum(NN_correct_right_2 ==1) + np.sum(NN_correct_left_2==1) + np.sum(NN_correct_right_2 ==0) + np.sum(NN_correct_left_2==0) )
NN_single_top_purity = ( np.sum(NN_correct_right_1 ==1) + np.sum(NN_correct_left_1==1)  )/( np.sum(NN_correct_right_1 ==1) + np.sum(NN_correct_left_1==1) + np.sum(NN_correct_right_1 ==0) + np.sum(NN_correct_left_1==0) )

print(f'Event putiry (no filter, NN)               : {NN_event_purity:.3f}')
print(f'Two top putiry (both matched, NN)          : {NN_two_top_purity:.3f}')
print(f'Single top putiry (partial matched, NN)    : {NN_single_top_purity:.3f}')

print("-"*80)

chi2_event_purity = np.sum(chi2_case==1)/( np.sum(chi2_case==1) + np.sum(chi2_case==2) + np.sum(chi2_case==3))
chi2_two_top_purity = ( np.sum(chi2_correct_right_2 ==1) + np.sum(chi2_correct_left_2==1)  )/( np.sum(chi2_correct_right_2 ==1) + np.sum(chi2_correct_left_2==1) + np.sum(chi2_correct_right_2 ==0) + np.sum(chi2_correct_left_2==0) )
chi2_single_top_purity = ( np.sum(chi2_correct_right_1 ==1) + np.sum(chi2_correct_left_1==1)  )/( np.sum(chi2_correct_right_1 ==1) + np.sum(chi2_correct_left_1==1) + np.sum(chi2_correct_right_1 ==0) + np.sum(chi2_correct_left_1==0) )
print(f'Event putiry (no filter, chi2)             : {chi2_event_purity:.3f}')
print(f'Two top putiry (both matched, chi2)        : {chi2_two_top_purity:.3f}')
print(f'Single top putiry (partial matched, chi2)  : {chi2_single_top_purity:.3f}')

Event putiry (no filter, NN)               : 0.867
Two top putiry (both matched, NN)          : 0.909
Single top putiry (partial matched, NN)    : 0.248
--------------------------------------------------------------------------------
Event putiry (no filter, chi2)             : 0.398
Two top putiry (both matched, chi2)        : 0.491
Single top putiry (partial matched, chi2)  : 0.156


In [19]:
NN_predict_6jet, NN_target_6jet = [], []
NN_predict_6jet_1, NN_target_6jet_1 = [], []
NN_predict_6jet_2, NN_target_6jet_2 = [], []

NN_predict_7jet, NN_target_7jet = [], []
NN_predict_7jet_1, NN_target_7jet_1 = [], []
NN_predict_7jet_2, NN_target_7jet_2 = [], []

NN_predict_8jet, NN_target_8jet = [], []
NN_predict_8jet_1, NN_target_8jet_1 = [], []
NN_predict_8jet_2, NN_target_8jet_2 = [], []
for i in range(len(full_predictions)):
    if num_jets[i] == 6:
        NN_predict_6jet.append(full_predictions[i])
        NN_target_6jet.append(full_targets[i])  
        if n_match[i] == 2:
            NN_predict_6jet_2.append(full_predictions[i])
            NN_target_6jet_2.append(full_targets[i])   
        elif n_match[i] == 1:
            NN_predict_6jet_1.append(full_predictions[i])
            NN_target_6jet_1.append(full_targets[i]) 
            
    elif num_jets[i] == 7:
        NN_predict_7jet.append(full_predictions[i])
        NN_target_7jet.append(full_targets[i])  
        if n_match[i] == 2:
            NN_predict_7jet_2.append(full_predictions[i])
            NN_target_7jet_2.append(full_targets[i])   
        elif n_match[i] == 1:
            NN_predict_7jet_1.append(full_predictions[i])
            NN_target_7jet_1.append(full_targets[i]) 
    elif num_jets[i] >= 8:
        NN_predict_8jet.append(full_predictions[i])
        NN_target_8jet.append(full_targets[i])  
        if n_match[i] == 2:
            NN_predict_8jet_2.append(full_predictions[i])
            NN_target_8jet_2.append(full_targets[i])   
        elif n_match[i] == 1:
            NN_predict_8jet_1.append(full_predictions[i])
            NN_target_8jet_1.append(full_targets[i]) 
            
chi2_predict_6jet, chi2_target_6jet = [], []
chi2_predict_6jet_1, chi2_target_6jet_1 = [], []
chi2_predict_6jet_2, chi2_target_6jet_2 = [], []

chi2_predict_7jet, chi2_target_7jet = [], []
chi2_predict_7jet_1, chi2_target_7jet_1 = [], []
chi2_predict_7jet_2, chi2_target_7jet_2 = [], []

chi2_predict_8jet, chi2_target_8jet = [], []
chi2_predict_8jet_1, chi2_target_8jet_1 = [], []
chi2_predict_8jet_2, chi2_target_8jet_2 = [], []
for i in range(len(chi2_result)):
    if len(jet_pt[i]) == 6:
        chi2_predict_6jet.append(chi2_result[i])
        chi2_target_6jet.append(delta_R_result[i])  
        if N_match_top_in_event[i] == 1:
            chi2_predict_6jet_1.append(chi2_result[i])
            chi2_target_6jet_1.append(delta_R_result[i])   
        elif N_match_top_in_event[i] == 2:
            chi2_predict_6jet_2.append(chi2_result[i])
            chi2_target_6jet_2.append(delta_R_result[i]) 
            
    elif len(jet_pt[i]) == 7:
        chi2_predict_7jet.append(chi2_result[i])
        chi2_target_7jet.append(delta_R_result[i])  
        if N_match_top_in_event[i] == 2:
            chi2_predict_7jet_2.append(chi2_result[i])
            chi2_target_7jet_2.append(delta_R_result[i])   
        elif N_match_top_in_event[i] == 1:
            chi2_predict_7jet_1.append(chi2_result[i])
            chi2_target_7jet_1.append(delta_R_result[i]) 
    elif len(jet_pt[i]) >= 8:
        chi2_predict_8jet.append(chi2_result[i])
        chi2_target_8jet.append(delta_R_result[i])  
        if N_match_top_in_event[i] == 2:
            chi2_predict_8jet_2.append(chi2_result[i])
            chi2_target_8jet_2.append(delta_R_result[i])   
        elif N_match_top_in_event[i] == 1:
            chi2_predict_8jet_1.append(chi2_result[i])
            chi2_target_8jet_1.append(delta_R_result[i]) 

In [20]:
NN_case_6jet, NN_correct_right_6jet, NN_correct_left_6jet = compute(NN_predict_6jet, NN_target_6jet)
NN_case_1_6jet, NN_correct_right_1_6jet, NN_correct_left_1_6jet = compute(NN_predict_6jet_1, NN_target_6jet_1)
NN_cas_2_6jet, NN_correct_right_2_6jet, NN_correct_left_2_6jet = compute(NN_predict_6jet_2, NN_target_6jet_2)

chi2_case_6jet, chi2_correct_right_6jet, chi2_correct_left_6jet = compute(chi2_predict_6jet, chi2_target_6jet)
chi2_case_1_6jet, chi2_correct_right_1_6jet, chi2_correct_left_1_6jet = compute(chi2_predict_6jet_1, chi2_target_6jet_1)
chi2_cas_2_6jet, chi2_correct_right_2_6jet, chi2_correct_left_2_6jet = compute(chi2_predict_6jet_2, chi2_target_6jet_2)

100%|██████████| 88373/88373 [00:02<00:00, 42314.74it/s]


In [32]:
NN_event_purity_6jet = np.sum(NN_case_6jet==1)/( np.sum(NN_case_6jet==1) + np.sum(NN_case_6jet==2) + np.sum(NN_case_6jet==3))
NN_two_top_purity_6jet = ( np.sum(NN_correct_right_2_6jet ==1) + np.sum(NN_correct_left_2_6jet==1)  )/( np.sum(NN_correct_right_2_6jet ==1) + np.sum(NN_correct_left_2_6jet==1) + np.sum(NN_correct_right_2_6jet ==0) + np.sum(NN_correct_left_2_6jet==0) )
NN_single_top_purity_6jet = ( np.sum(NN_correct_right_1_6jet ==1) + np.sum(NN_correct_left_1_6jet==1)  )/( np.sum(NN_correct_right_1_6jet ==1) + np.sum(NN_correct_left_1_6jet==1) + np.sum(NN_correct_right_1_6jet ==0) + np.sum(NN_correct_left_1_6jet==0) )

print(f'Event putiry (no filter, NN)               : {NN_event_purity_6jet:.3f}')
print(f'Two top putiry (both matched, NN)          : {NN_two_top_purity_6jet:.3f}')
print(f'Single top putiry (partial matched, NN)    : {NN_single_top_purity_6jet:.3f}')

print("-"*80)

chi2_event_purity_6jet = np.sum(chi2_case_6jet==1)/( np.sum(chi2_case_6jet==1) + np.sum(chi2_case_6jet==2) + np.sum(chi2_case_6jet==3))
chi2_two_top_purity_6jet =  ( np.sum(chi2_correct_right_2_6jet ==1) + np.sum(chi2_correct_left_2_6jet==1)  )/( np.sum(chi2_correct_right_2_6jet ==1) + np.sum(chi2_correct_left_2_6jet==1) + np.sum(chi2_correct_right_2_6jet ==0) + np.sum(chi2_correct_left_2_6jet==0) )
chi2_single_top_purity_6jet = ( np.sum(chi2_correct_right_1_6jet ==1) + np.sum(chi2_correct_left_1_6jet==1)  )/( np.sum(chi2_correct_right_1_6jet ==1) + np.sum(chi2_correct_left_1_6jet==1) + np.sum(chi2_correct_right_1_6jet ==0) + np.sum(chi2_correct_left_1_6jet==0) )
print(f'Event putiry (no filter, chi2)             : {chi2_event_purity_6jet:.3f}')
print(f'Two top putiry (both matched, chi2)        : {chi2_two_top_purity_6jet:.3f}')
print(f'Single top putiry (partial matched, chi2)  : {chi2_single_top_purity_6jet:.3f}')

Event putiry (no filter, NN)               : 0.930
Two top putiry (both matched, NN)          : 0.948
Single top putiry (partial matched, NN)    : 0.271
--------------------------------------------------------------------------------
Event putiry (no filter, chi2)             : 0.638
Two top putiry (both matched, chi2)        : 0.668
Single top putiry (partial matched, chi2)  : 0.163


In [23]:
NN_case_7jet, NN_correct_right_7jet, NN_correct_left_7jet = compute(NN_predict_7jet, NN_target_7jet)
NN_case_1_7jet, NN_correct_right_1_7jet, NN_correct_left_1_7jet = compute(NN_predict_7jet_1, NN_target_7jet_1)
NN_cas_2_7jet, NN_correct_right_2_7jet, NN_correct_left_2_7jet = compute(NN_predict_7jet_2, NN_target_7jet_2)

chi2_case_7jet, chi2_correct_right_7jet, chi2_correct_left_7jet = compute(chi2_predict_7jet, chi2_target_7jet)
chi2_case_1_7jet, chi2_correct_right_1_7jet, chi2_correct_left_1_7jet = compute(chi2_predict_7jet_1, chi2_target_7jet_1)
chi2_cas_2_7jet, chi2_correct_right_2_7jet, chi2_correct_left_2_7jet = compute(chi2_predict_7jet_2, chi2_target_7jet_2)

100%|██████████| 126920/126920 [00:03<00:00, 36515.16it/s]


In [30]:
NN_event_purity_7jet = np.sum(NN_case_7jet==1)/( np.sum(NN_case_7jet==1) + np.sum(NN_case_7jet==2) + np.sum(NN_case_7jet==3))
NN_two_top_purity_7jet =( np.sum(NN_correct_right_2_7jet ==1) + np.sum(NN_correct_left_2_7jet==1)  )/( np.sum(NN_correct_right_2_7jet ==1) + np.sum(NN_correct_left_2_7jet==1) + np.sum(NN_correct_right_2_7jet ==0) + np.sum(NN_correct_left_2_7jet==0) )
NN_single_top_purity_7jet = ( np.sum(NN_correct_right_1_7jet ==1) + np.sum(NN_correct_left_1_7jet==1)  )/( np.sum(NN_correct_right_1_7jet ==1) + np.sum(NN_correct_left_1_7jet==1) + np.sum(NN_correct_right_1_7jet ==0) + np.sum(NN_correct_left_1_7jet==0) )

print(f'Event putiry (no filter, NN)               : {NN_event_purity_7jet:.3f}')
print(f'Two top putiry (both matched, NN)          : {NN_two_top_purity_7jet:.3f}')
print(f'Single top putiry (partial matched, NN)    : {NN_single_top_purity_7jet:.3f}')

print("-"*80)

chi2_event_purity_7jet = np.sum(chi2_case_7jet==1)/( np.sum(chi2_case_7jet==1) + np.sum(chi2_case_7jet==2) + np.sum(chi2_case_7jet==3))
chi2_two_top_purity_7jet = ( np.sum(chi2_correct_right_2_7jet ==1) + np.sum(chi2_correct_left_2_7jet==1)  )/( np.sum(chi2_correct_right_2_7jet ==1) + np.sum(chi2_correct_left_2_7jet==1) + np.sum(chi2_correct_right_2_7jet ==0) + np.sum(chi2_correct_left_2_7jet==0) )
chi2_single_top_purity_7jet = ( np.sum(chi2_correct_right_1_7jet ==1) + np.sum(chi2_correct_left_1_7jet==1)  )/( np.sum(chi2_correct_right_1_7jet ==1) + np.sum(chi2_correct_left_1_7jet==1) + np.sum(chi2_correct_right_1_7jet ==0) + np.sum(chi2_correct_left_1_7jet==0) )
print(f'Event putiry (no filter, chi2)             : {chi2_event_purity_7jet:.3f}')
print(f'Two top putiry (both matched, chi2)        : {chi2_two_top_purity_7jet:.3f}')
print(f'Single top putiry (partial matched, chi2)  : {chi2_single_top_purity_7jet:.3f}')

Event putiry (no filter, NN)               : 0.878
Two top putiry (both matched, NN)          : 0.916
Single top putiry (partial matched, NN)    : 0.251
--------------------------------------------------------------------------------
Event putiry (no filter, chi2)             : 0.432
Two top putiry (both matched, chi2)        : 0.526
Single top putiry (partial matched, chi2)  : 0.164


In [27]:
NN_case_8jet, NN_correct_right_8jet, NN_correct_left_8jet = compute(NN_predict_8jet, NN_target_8jet)
NN_case_1_8jet, NN_correct_right_1_8jet, NN_correct_left_1_8jet = compute(NN_predict_8jet_1, NN_target_8jet_1)
NN_cas_2_8jet, NN_correct_right_2_8jet, NN_correct_left_2_8jet = compute(NN_predict_8jet_2, NN_target_8jet_2)

chi2_case_8jet, chi2_correct_right_8jet, chi2_correct_left_8jet = compute(chi2_predict_8jet, chi2_target_8jet)
chi2_case_1_8jet, chi2_correct_right_1_8jet, chi2_correct_left_1_8jet = compute(chi2_predict_8jet_1, chi2_target_8jet_1)
chi2_cas_2_8jet, chi2_correct_right_2_8jet, chi2_correct_left_2_8jet = compute(chi2_predict_8jet_2, chi2_target_8jet_2)

100%|██████████| 171738/171738 [00:05<00:00, 30925.00it/s]

Event putiry (no filter, NN)               : 0.826
Two top putiry (both matched, NN)          : 0.876
Single top putiry (partial matched, NN)    : 0.224
--------------------------------------------------------------------------------
Event putiry (no filter, chi2)             : 0.250
Two top putiry (both matched, chi2)        : 0.285
Single top putiry (partial matched, chi2)  : 0.133


In [29]:
NN_event_purity_8jet = np.sum(NN_case_8jet==1)/( np.sum(NN_case_8jet==1) + np.sum(NN_case_8jet==2) + np.sum(NN_case_8jet==3))
NN_two_top_purity_8jet = ( np.sum(NN_correct_right_2_8jet ==1) + np.sum(NN_correct_left_2_8jet==1)  )/( np.sum(NN_correct_right_2_8jet ==1) + np.sum(NN_correct_left_2_8jet==1) + np.sum(NN_correct_right_2_8jet ==0) + np.sum(NN_correct_left_2_8jet==0) )
NN_single_top_purity_8jet = ( np.sum(NN_correct_right_1_8jet ==1) + np.sum(NN_correct_left_1_8jet==1)  )/( np.sum(NN_correct_right_1_8jet ==1) + np.sum(NN_correct_left_1_8jet==1) + np.sum(NN_correct_right_1_8jet ==0) + np.sum(NN_correct_left_1_8jet==0) )

print(f'Event putiry (no filter, NN)               : {NN_event_purity_8jet:.3f}')
print(f'Two top putiry (both matched, NN)          : {NN_two_top_purity_8jet:.3f}')
print(f'Single top putiry (partial matched, NN)    : {NN_single_top_purity_8jet:.3f}')

print("-"*80)

chi2_event_purity_8jet = np.sum(chi2_case_8jet==1)/( np.sum(chi2_case_8jet==1) + np.sum(chi2_case_8jet==2) + np.sum(chi2_case_8jet==3))
chi2_two_top_purity_8jet = ( np.sum(chi2_correct_right_2_8jet ==1) + np.sum(chi2_correct_left_2_8jet==1)  )/( np.sum(chi2_correct_right_2_8jet ==1) + np.sum(chi2_correct_left_2_8jet==1) + np.sum(chi2_correct_right_2_8jet ==0) + np.sum(chi2_correct_left_2_8jet==0) )
chi2_single_top_purity_8jet = ( np.sum(chi2_correct_right_1_8jet ==1) + np.sum(chi2_correct_left_1_8jet==1)  )/( np.sum(chi2_correct_right_1_8jet ==1) + np.sum(chi2_correct_left_1_8jet==1) + np.sum(chi2_correct_right_1_8jet ==0) + np.sum(chi2_correct_left_1_8jet==0) )
print(f'Event putiry (no filter, chi2)             : {chi2_event_purity_8jet:.3f}')
print(f'Two top putiry (both matched, chi2)        : {chi2_two_top_purity_8jet:.3f}')
print(f'Single top putiry (partial matched, chi2)  : {chi2_single_top_purity_8jet:.3f}')

Event putiry (no filter, NN)               : 0.826
Two top putiry (both matched, NN)          : 0.883
Single top putiry (partial matched, NN)    : 0.224
--------------------------------------------------------------------------------
Event putiry (no filter, chi2)             : 0.250
Two top putiry (both matched, chi2)        : 0.375
Single top putiry (partial matched, chi2)  : 0.133


# Alex's result(using Alex's script)<br>
Total number of correct top quarks   : 983052<br>
Total number of incorrect top quarks : 410606<br>
Total number of incomplete top quarks: 974854<br>
<br>
Top Purity (Unfiltered): 0.237<br>
Top Purity (1 Top Events): 0.293<br>
Top Purity (2 Top Events): 0.491<br>
<br>
Event Purity (Unfiltered): 0.130<br>
Event Purity (2 Top Events): 0.398<br>
<br>
Total ratio of 0 top events: 0.150<br>
Total ratio of 1 top events: 0.523<br>
Total ratio of 2 top events: 0.327<br>
<br>
--------------------------------------------------------------------------------<br>
Top Purity (1 Top Events): 0.293<br>
Top Purity (2 Top Events): 0.491<br>
Event Purity (2 Top Events): 0.398<br>